# Résolution de Sudoku avec OR-Tools

## Introduction

Google OR-Tools est une suite de logiciels d'optimisation développée par Google. Elle permet de résoudre des problèmes complexes d'optimisation combinatoire tels que la satisfaction de contraintes (CSP), la programmation linéaire (LP), la programmation linéaire mixte (MIP), et bien plus. Dans ce projet, nous nous concentrerons sur la résolution de Sudokus en utilisant différentes techniques fournies par OR-Tools.

### Types de Solveurs

1. **Solveur de Satisfaction de Contraintes (CSP)** :
   Ce solveur utilise des techniques de propagation de contraintes et de recherche pour trouver des solutions qui satisfont toutes les contraintes spécifiées.

2. **Solveur de Programmation Linéaire Mixte (MIP)** :
   Ce solveur combine la programmation linéaire avec des variables entières pour résoudre des problèmes d'optimisation. Il est particulièrement efficace pour les problèmes où certaines variables doivent être entières.

Nous allons explorer ces deux solveurs pour résoudre des grilles de Sudoku et comparer leurs performances.

## Installation de OR-Tools

Pour utiliser OR-Tools avec C#, nous devons d'abord installer la bibliothèque. Vous pouvez suivre les instructions sur le site officiel de [Google OR-Tools](https://developers.google.com/optimization).

### Installation de OR-Tools


In [17]:
#r "nuget: Google.OrTools"

Installed Packages Google.OrTools, 9.10.4067

## Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [18]:
#!import Sudoku-0-Environment.ipynb

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger et afficher des grilles de Sudoku.


## Affichage des Puzzles de chaque Difficulté

Nous allons charger et afficher un puzzle de chaque niveau de difficulté : Facile, Moyen et Difficile.


Puzzle Facile:
-------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Puzzle Moyen:
-------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Puzzle Difficile:
-------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

## Implémentations des Solveurs

### Solveur par contraintes (CpSolver)

L'algorithme de satisfaction de contraintes (CSP) utilise les contraintes pour réduire l'espace de recherche et trouver une solution qui satisfait toutes les contraintes du problème. Nous utilisons `CpSolver` d'OR-Tools pour cette implémentation.


In [19]:
using Google.OrTools.ConstraintSolver;
using SimpleCPSolver = Google.OrTools.ConstraintSolver.Solver;
using SimpleConstraint = Google.OrTools.ConstraintSolver.Constraint;

public class OrToolsCpSolver : ISudokuSolver
{
    private const int GridSize = 9;

    public SudokuGrid Solve(SudokuGrid s)
    {
        var solver = new SimpleCPSolver("CpSimple");

        IntVar[,] gridVars = new IntVar[GridSize, GridSize];
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                gridVars[i, j] = s.Cells[i, j] == 0 
                                 ? solver.MakeIntVar(1, GridSize, $"Cell({i},{j})") 
                                 : solver.MakeIntConst(s.Cells[i, j]);
            }
        }


        AddAllDifferentConstraints(solver, gridVars);

        DecisionBuilder db = solver.MakePhase(gridVars.Flatten(), SimpleCPSolver.CHOOSE_FIRST_UNBOUND, SimpleCPSolver.ASSIGN_MIN_VALUE);

        solver.NewSearch(db);
        while (solver.NextSolution())
        {
            for (int i = 0; i < GridSize; i++)
            {
                for (int j = 0; j < GridSize; j++)
                {
                    s.Cells[i, j] = (int)gridVars[i, j].Value();
                }
            }
        }

        return s;
    }

   private void AddAllDifferentConstraints(SimpleCPSolver solver, IntVar[,] gridVars)
    {
        for (int i = 0; i < GridSize; i++)
        {
            solver.Add(AllDifferent(solver, GetRow(gridVars, i)));
            solver.Add(AllDifferent(solver, GetColumn(gridVars, i)));
        }

        for (int i = 0; i < GridSize; i += 3)
        {
            for (int j = 0; j < GridSize; j += 3)
            {
                solver.Add(AllDifferent(solver, GetSubGrid(gridVars, i, j)));
            }
        }
    }

    private IntVar[] GetRow(IntVar[,] matrix, int row)
    {
        IntVar[] result = new IntVar[GridSize];
        for (int col = 0; col < GridSize; col++)
        {
            result[col] = matrix[row, col];
        }
        return result;
    }

    private IntVar[] GetColumn(IntVar[,] matrix, int col)
    {
        IntVar[] result = new IntVar[GridSize];
        for (int row = 0; row < GridSize; row++)
        {
            result[row] = matrix[row, col];
        }
        return result;
    }

    private IntVar[] GetSubGrid(IntVar[,] matrix, int startRow, int startCol)
    {
        IntVar[] result = new IntVar[GridSize];
        int index = 0;
        for (int i = 0; i < 3; i++)
        {
            for (int j = 0; j < 3; j++)
            {
                result[index++] = matrix[startRow + i, startCol + j];
            }
        }
        return result;
    }

    private SimpleConstraint AllDifferent(SimpleCPSolver solver, IntVar[] vars)
    {
        return solver.MakeAllDifferent(vars);
    }
}

### Tests du Solveur par contraintes

Nous allons tester ce solveur sur une grille de Sudoku.

In [20]:
var cpSolver = new OrToolsCpSolver();

var easySudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Easy).FirstOrDefault();
Console.WriteLine("Puzzle Sudoku Facile Initial:");
SudokuHelper.SolveSudoku(easySudoku, cpSolver);

var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).FirstOrDefault();
Console.WriteLine("Puzzle Sudoku Moyen Initial:");
SudokuHelper.SolveSudoku(mediumSudoku, cpSolver);

var hardSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Hard).FirstOrDefault();
Console.WriteLine("Puzzle Sudoku Difficile Initial:");
SudokuHelper.SolveSudoku(hardSudoku, cpSolver);


Puzzle Sudoku Facile Initial:


Résolution par le solver OrToolsCpSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 1,8912 ms

Puzzle Sudoku Moyen Initial:


Résolution par le solver OrToolsCpSolver du Sudoku:
 -------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 8  5  9 | 6  1  2 | 4  3  7 | 
| 7  2  3 | 8  5  4 | 1  6  9 | 
| 1  6  4 | 3  7  9 | 5  2  8 | 
-------------------------------
| 9  8  6 | 1  4  7 | 3  5  2 | 
| 3  7  5 | 2  6  8 | 9  1  4 | 
| 2  4  1 | 5  9  3 | 7  8  6 | 
-------------------------------
| 4  3  2 | 9  8  1 | 6  7  5 | 
| 6  1  7 | 4  2  5 | 8  9  3 | 
| 5  9  8 | 7  3  6 | 2  4  1 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 2,0463 ms

Puzzle Sudoku Difficile Initial:


Résolution par le solver OrToolsCpSolver du Sudoku:
 -------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 4  1  7 | 3  6  9 | 8  2  5 | 
| 6  3  2 | 1  5  8 | 9  4  7 | 
| 9  5  8 | 7  2  4 | 3  1  6 | 
-------------------------------
| 8  2  5 | 4  3  7 | 1  6  9 | 
| 7  9  1 | 5  8  6 | 4  3  2 | 
| 3  4  6 | 9  1  2 | 7  5  8 | 
-------------------------------
| 2  8  9 | 6  4  3 | 5  7  1 | 
| 5  7  3 | 2  9  1 | 6  8  4 | 
| 1  6  4 | 8  7  5 | 2  9  3 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 27,1 ms

### Solveur MIP (Mixed-Integer Programming)

L'algorithme de programmation linéaire mixte (MIP) combine la programmation linéaire et les variables entières pour résoudre des problèmes d'optimisation. Nous utilisons `Mixed-Integer Programming` d'OR-Tools pour cette implémentation.


In [21]:
using Google.OrTools.LinearSolver;
using LinearSolver = Google.OrTools.LinearSolver.Solver;

public class OrToolsMIPSolver : ISudokuSolver
{
    private const int GridSize = 9;

    public SudokuGrid Solve(SudokuGrid s)
    {
        LinearSolver solver = LinearSolver.CreateSolver("SCIP");
        if (solver == null)
        {
            throw new InvalidOperationException("Solver initialization failed.");
        }

        Variable[,,] cells = new Variable[GridSize, GridSize, GridSize];
        InitializeVariables(s, solver, cells);

        AddConstraints(solver, cells);

        if (solver.Solve() != LinearSolver.ResultStatus.OPTIMAL)
        {
            throw new Exception("No solution found.");
        }

        return ExtractSolution(s, cells);
    }

    private void InitializeVariables(SudokuGrid s, LinearSolver solver, Variable[,,] cells)
    {
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                for (int k = 0; k < GridSize; k++)
                {
                    cells[i, j, k] = solver.MakeIntVar(0, 1, $"Cell({i},{j},{k})");
                }
                if (s.Cells[i, j] != 0)
                {
                    solver.Add(cells[i, j, s.Cells[i, j] - 1] == 1);
                }
            }
        }
    }

    private void AddConstraints(LinearSolver solver, Variable[,,] cells)
    {
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                solver.Add(Sum(solver, Enumerable.Range(0, GridSize).Select(k => cells[i, j, k])) == 1);
            }
        }

        for (int k = 0; k < GridSize; k++)
        {
            for (int i = 0; i < GridSize; i++)
            {
                solver.Add(Sum(solver, Enumerable.Range(0, GridSize).Select(j => cells[i, j, k])) == 1);
                solver.Add(Sum(solver, Enumerable.Range(0, GridSize).Select(j => cells[j, i, k])) == 1);
            }
        }

        for (int k = 0; k < GridSize; k++)
        {
            for (int i = 0; i < GridSize; i += 3)
            {
                for (int j = 0; j < GridSize; j += 3)
                {
                    solver.Add(Sum(solver, Enumerable.Range(0, 3).SelectMany(row => Enumerable.Range(0, 3).Select(col => cells[i + row, j + col, k]))) == 1);
                }
            }
        }
    }

    private LinearExpr Sum(LinearSolver solver, IEnumerable<Variable> vars)
    {
        LinearExpr sum = new();
        foreach (var v in vars)
        {
            sum += v;
        }
        return sum;
    }

    private SudokuGrid ExtractSolution(SudokuGrid s, Variable[,,] cells)
    {
        SudokuGrid solution = new SudokuGrid();
        for (int i = 0; i < GridSize; i++)
        {
            for (int j = 0; j < GridSize; j++)
            {
                for (int k = 0; k < GridSize; k++)
                {
                    if (cells[i, j, k].SolutionValue() == 1)
                    {
                        solution.Cells[i, j] = k + 1;
                    }
                }
            }
        }
        return solution;
    }
}



### Tests des Solveurs

Nous allons maintenant tester les solveurs implémentés sur des grilles de Sudoku de différentes difficultés : Facile, Moyen et Difficile. Nous mesurerons également le temps de résolution et vérifierons la validité des solutions trouvées.


In [23]:
var cpSolver = new OrToolsCpSolver();
var mipSolver = new OrToolsMIPSolver();

void TestSolver(ISudokuSolver solver, SudokuDifficulty difficulty, string solverName)
{
    var sudoku = SudokuHelper.GetSudokus(difficulty).FirstOrDefault();
    Console.WriteLine($"Résolution par le {solverName} du Sudoku {difficulty}:");
    SudokuHelper.SolveSudoku(sudoku, solver);
}

TestSolver(cpSolver, SudokuDifficulty.Easy, "solver OrToolsCpSolver");
TestSolver(mipSolver, SudokuDifficulty.Easy, "solver OrToolsMIPSolver (SCIP)");
TestSolver(cpSolver, SudokuDifficulty.Medium, "solver OrToolsCpSolver");
TestSolver(mipSolver, SudokuDifficulty.Medium, "solver OrToolsMIPSolver (SCIP)");
TestSolver(cpSolver, SudokuDifficulty.Hard, "solver OrToolsCpSolver");
TestSolver(mipSolver, SudokuDifficulty.Hard, "solver OrToolsMIPSolver (SCIP)");


Résolution par le solver OrToolsCpSolver du Sudoku Easy:


Résolution par le solver OrToolsCpSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 1,0297 ms

Résolution par le solver OrToolsMIPSolver (SCIP) du Sudoku Easy:


Résolution par le solver OrToolsMIPSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

...

Résolution par le solver OrToolsCpSolver du Sudoku:
 -------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 4  1  7 | 3  6  9 | 8  2  5 | 
| 6  3  2 | 1  5  8 | 9  4  7 | 
| 9  5  8 | 7  2  4 | 3  1  6 | 
-------------------------------
| 8  2  5 | 4  3  7 | 1  6  9 | 
| 7  9  1 | 5  8  6 | 4  3  2 | 
| 3  4  6 | 9  1  2 | 7  5  8 | 
-------------------------------
| 2  8  9 | 6  4  3 | 5  7  1 | 
| 5  7  3 | 2  9  1 | 6  8  4 | 
| 1  6  4 | 8  7  5 | 2  9  3 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 29,8822 ms

Résolution par le solver OrToolsMIPSolver (SCIP) du Sudoku Hard:


Résolution par le solver OrToolsMIPSolver du Sudoku:
 -------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

Sudoku renvoyé:
-------------------------------
| 4  1  7 | 3  6  9 | 8  2  5 | 
| 6  3  2 | 1  5  8 | 9  4  7 | 
| 9  5  8 | 7  2  4 | 3  1  6 | 
-------------------------------
| 8  2  5 | 4  3  7 | 1  6  9 | 
| 7  9  1 | 5  8  6 | 4  3  2 | 
| 3  4  6 | 9  1  2 | 7  5  8 | 
-------------------------------
| 2  8  9 | 6  4  3 | 5  7  1 | 
| 5  7  3 | 2  9  1 | 6  8  4 | 
| 1  6  4 | 8  7  5 | 2  9  3 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 10,5879 ms